В командной строке:

- pip install zeyrek
- pip3 install NlpToolkit-MorphologicalAnalysis
- pip install nltk
- pip install corpy

В интерактивной среде питона:

    import nltk
    nltk.download()
    
(Это нужно выполнить только один раз для загрузки необходимых вещей. Выползет окошечко, в котором нужно выбрать "all" и загрузить). 

In [60]:
# импортируем нужные модули

import pandas as pd
import json
import zeyrek
import re
from MorphologicalAnalysis import FsmMorphologicalAnalyzer
from corpy.udpipe import Model

In [41]:
data = []
with open('turk2.json', 'r',  encoding='utf8') as file:  # здесь нужно поменять имя файла на свое
    for line in file:
        data.append(json.loads(line))
        
data = pd.DataFrame(data)

In [42]:
data.head()  # теперь у нас с вами есть датасет пандас, в котором могут быть полезные колонки (скорее всего, пригодится колонка "language", чтобы отсеять нетурецкие твиты). 

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1448655827259199488,1448655827259199488,2021-10-14 17:23:58 MSK,2021-10-14,17:23:58,+0300,109226167,mevlutcavusoglu,Mevlüt Çavuşoğlu,,...,,,,,,[],,,,
1,1448616241510354949,1448616241510354949,2021-10-14 14:46:40 MSK,2021-10-14,14:46:40,+0300,109226167,mevlutcavusoglu,Mevlüt Çavuşoğlu,,...,,,,,,[],,,,
2,1448616185965199368,1448616185965199368,2021-10-14 14:46:27 MSK,2021-10-14,14:46:27,+0300,109226167,mevlutcavusoglu,Mevlüt Çavuşoğlu,,...,,,,,,[],,,,
3,1448234252097789954,1448234252097789954,2021-10-13 13:28:47 MSK,2021-10-13,13:28:47,+0300,109226167,mevlutcavusoglu,Mevlüt Çavuşoğlu,,...,,,,,,[],,,,
4,1448234204706349059,1448234204706349059,2021-10-13 13:28:35 MSK,2021-10-13,13:28:35,+0300,109226167,mevlutcavusoglu,Mevlüt Çavuşoğlu,,...,,,,,,[],,,,


In [43]:
data.language.unique()  # на каких языках есть твиты?

array(['tr', 'en', 'es', 'ro', 'lv', 'und', 'de', 'ja', 'ar', 'ru', 'pl',
       'fr', 'ht', 'fa', 'in', 'et', 'it', 'pt', 'eu', 'fi', 'da', 'lt',
       'ur', 'tl', 'el', 'ko', 'ca', 'no', 'hu', 'ka', 'nl', 'hi', 'sv',
       'cy'], dtype=object)

In [44]:
data = data[data.language == 'tr']  # отфильтруем только турецкий

In [45]:
data.tweet = data.tweet.apply(lambda tweet: re.sub(r'https://.+|@\S+|#\S+', '', tweet))
# почистим твиты от адресов, обращений к людям (@google) и хештегов

In [46]:
pd.set_option('display.max_colwidth', None)  # эта штука нужна, чтобы отображались твиты целиком
data[['tweet']].head()

,tweet
0,Geçici Hükümeti Dışişleri Bakan Vekili Amir Khan Muttaqi ile Görüşme Sonrası Basını Bilgilendirme 📍Ankara
1,"Bugünkü ilk konuğumuz kardeş Dışişleri Bakanı Najla Mangoush ile ikili ilişkilerimizi ve gündemimizdeki konuları ele aldık. -Seçimlerin özgür, adil ve güvenilir şekilde yapılmasını destekliyoruz. -Libya'nın istikrar ve refahına katkılarımızı sürdüreceğiz.🇹🇷🇱🇾"
3,"Bu hafta Latin Amerika'dan ağırladığımız üçüncü konuğumuz DB Denis Moncada’ydı. -Eğitim, Bakanlıklarımız arasında siyasi istişareler ve protokol anlaşmalarını imzaladık. -Tarım ve inşaat alanlarında işbirliği yapacağız. -Büyükelçiliklerimizi karşılıklı açacağız.🇹🇷🇳🇮"
5,"Dışişleri Bakanı Denis Moncada Colindres ile Ortak Basın Toplantısı Conferencia de Prensa Conjunta con Denis Moncada Colindres, Ministro de Relaciones Exteriores de 🇹🇷🇳🇮 📍Ankara"
6,"Kardeş Temsilciler Meclisi Başkanıyla görüştük. 2022’de diplomatik ilişkilerimizin 30. yıldönümünü birlikte kutlayacağız. Met w/Zokirzoda, Speaker of brotherly House of Representatives. In 2022 will celebrate 30th anniv. of diplomatic relations.🇹🇷🇹🇯"


Правиловый парсер zeyrek

In [61]:
token = re.compile(r'''(?i)\b[a-zçğıüöş]+(?:-[a-zçğıüöş]+)*\b|[.,!@#$%\^&*()\[\]{};:'"?><\\/`~“”]+''') # скомпилируем регулярку для токенизации

In [47]:
tokens = []
for elem in data.tweet:
    tokens.extend(re.findall(token, elem))
tokens[:10]

['Geçici',
 'Hükümeti',
 'Dışişleri',
 'Bakan',
 'Vekili',
 'Amir',
 'Khan',
 'Muttaqi',
 'ile',
 'Görüşme']

In [48]:
analyzer = zeyrek.MorphAnalyzer()

In [49]:
res = []
for token in tokens[:1000]:  # обратите внимание, я только на первой тысяче токенов делаю
    res.append(analyzer.analyze(token))
res[:10]

[[[Parse(word='Geçici', lemma='geçici', pos='Adj', morphemes=['Adj'], formatted='[geçici:Adj] geçici:Adj'),
   Parse(word='Geçici', lemma='geçmek', pos='Adj', morphemes=['Verb', 'Agt', 'Adj'], formatted='[geçmek:Verb] geç:Verb|ici:Agt→Adj'),
   Parse(word='Geçici', lemma='geçici', pos='Noun', morphemes=['Noun', 'A3sg'], formatted='[geçici:Noun] geçici:Noun+A3sg'),
   Parse(word='Geçici', lemma='geçmek', pos='Noun', morphemes=['Verb', 'Agt', 'Noun', 'A3sg'], formatted='[geçmek:Verb] geç:Verb|ici:Agt→Noun+A3sg')]],
 [[Parse(word='Hükümeti', lemma='hükümet', pos='Noun', morphemes=['Noun', 'A3sg', 'Acc'], formatted='[hükümet:Noun] hükümet:Noun+A3sg+i:Acc'),
   Parse(word='Hükümeti', lemma='hükümet', pos='Noun', morphemes=['Noun', 'A3sg', 'P3sg'], formatted='[hükümet:Noun] hükümet:Noun+A3sg+i:P3sg')]],
 [[Parse(word='Dışişleri', lemma='dışiş', pos='Noun', morphemes=['Noun', 'A3sg', 'P3pl'], formatted='[dışiş:Noun] dışiş:Noun+A3sg+leri:P3pl'),
   Parse(word='Dışişleri', lemma='dışiş', pos='N

Очевидно, zeyrek устроен примерно так же, как и pymorphy2 (скоро будем разбирать его). Значит, чтобы получить наиболее вероятные разборы, нужно в списке парсов выбрать первый парс. Типа того:

In [50]:
zeyrekresults = [{'word': token[0][0].word, 
                  'lemma': token[0][0].lemma, 
                  'pos': token[0][0].morphemes[0],
                  'morph': ','.join(token[0][0].morphemes[1:])} for token in res if token]

# для чего я так сложно сделала?..

In [51]:
zeyrekresults[:2]

[{'word': 'Geçici', 'lemma': 'geçici', 'pos': 'Adj', 'morph': ''},
 {'word': 'Hükümeti', 'lemma': 'hükümet', 'pos': 'Noun', 'morph': 'A3sg,Acc'}]

In [52]:
# ...а чтобы хоба:
zeyrekpd = pd.DataFrame(zeyrekresults)
zeyrekpd.head()

,word,lemma,pos,morph
0,Geçici,geçici,Adj,
1,Hükümeti,hükümet,Noun,"A3sg,Acc"
2,Dışişleri,dışiş,Noun,"A3sg,P3pl"
3,Bakan,bakmak,Verb,"PresPart,Adj"
4,Vekili,Vekil,Noun,"A3sg,Acc"


In [53]:
# а эту хобу уже можно в эксель записать:
zeyrekpd.to_excel('zeyrekmorph.xlsx')

Правиловый парсер номер два

In [54]:
fsm = FsmMorphologicalAnalyzer.FsmMorphologicalAnalyzer()

In [55]:
res = []
for token in tokens[:1000]:
    parse = fsm.morphologicalAnalysis(token)
    s = [token]
    for i in range(parse.size()):
        s.append(parse.getFsmParse(i).transitionList())
    res.append(s)
res[:10]

[['Geçici',
  'geç+VERB+POS^DB+ADJ+AGT',
  'geç+VERB+POS^DB+NOUN+AGT+A3SG+PNON+NOM',
  'geçici+ADJ'],
 ['Hükümeti', 'hükümet+NOUN+A3SG+P3SG+NOM', 'hükümet+NOUN+A3SG+PNON+ACC'],
 ['Dışişleri', 'dışişleri+NOUN+A3PL+PNON+NOM'],
 ['Bakan', 'bak+VERB+POS^DB+ADJ+PRESPART', 'bakan+NOUN+A3SG+PNON+NOM'],
 ['Vekili', 'vekil+NOUN+A3SG+P3SG+NOM', 'vekil+NOUN+A3SG+PNON+ACC'],
 ['Amir', 'amir+NOUN+A3SG+PNON+NOM', 'amir+NOUN+PROP+A3SG+PNON+NOM'],
 ['Khan'],
 ['Muttaqi'],
 ['ile', 'il+NOUN+A3SG+PNON+DAT', 'ile+CONJ', 'ile+POSTP+PCNOM'],
 ['Görüşme',
  'gör+VERB+RECIP+NEG+IMP+A2SG',
  'gör+VERB+RECIP+POS^DB+NOUN+INF2+A3SG+PNON+NOM',
  'görüş+VERB+NEG+IMP+A2SG',
  'görüş+VERB+POS^DB+NOUN+INF2+A3SG+PNON+NOM',
  'görüşme+NOUN+A3SG+PNON+NOM']]

Вторая хоба мне меньше нравится, но в целом можно и ее разодрать:

In [58]:
fsmresults = []
for token in res:
    if len(token) > 1:
        dct = {'word': token[0]}
        dissect = token[1].split('+')
        dct['stem'] = dissect[0]
        dct['pos'] = dissect[1]
        dct['morph'] = ', '.join(dissect[2:])
        fsmresults.append(dct)
    else:
        fsmresults.append({'word': token[0], 'stem': token[0], 'pos': None, 'morph': None})
fsmresults = pd.DataFrame(fsmresults)
fsmresults.head(10)

,word,stem,pos,morph
0,Geçici,geç,VERB,"POS^DB, ADJ, AGT"
1,Hükümeti,hükümet,NOUN,"A3SG, P3SG, NOM"
2,Dışişleri,dışişleri,NOUN,"A3PL, PNON, NOM"
3,Bakan,bak,VERB,"POS^DB, ADJ, PRESPART"
4,Vekili,vekil,NOUN,"A3SG, P3SG, NOM"
5,Amir,amir,NOUN,"A3SG, PNON, NOM"
6,Khan,Khan,None,None
7,Muttaqi,Muttaqi,None,None
8,ile,il,NOUN,"A3SG, PNON, DAT"
9,Görüşme,gör,VERB,"RECIP, NEG, IMP, A2SG"


In [59]:
fsmresults.to_excel('fsmmorph.xlsx')

А теперь udpipe для турецкого

In [43]:
m = Model('turkish-imst-ud-2.5-191206.udpipe')  # загрузим модель udpipe для турецкого языка

In [44]:
sents = list(m.process(data.loc[0, 'tweet'])) # это тестовый прогон на единственном твите

In [45]:
for sent in sents:
    for word in sent.words:
        print(word.form, word.lemma, word.feats)

<root> <root> <root>
Geçici geçici 
Hükümeti hükümet Case=Nom|Number=Sing|Number[psor]=Sing|Person=3|Person[psor]=3
Dışişleri dışişleri Case=Nom|Number=Plur|Person=3
Bakan bakan Case=Nom|Number=Sing|Person=3
Vekili Vekili Case=Nom|Number=Sing|Number[psor]=Sing|Person=3|Person[psor]=3
Amir amir Case=Nom|Number=Sing|Person=3
Khan Khan Case=Nom|Number=Sing|Person=3
Muttaqi Muttaq Case=Nom|Number=Sing|Person=3
ile ile 
Görüşme görüşme Case=Nom|Number=Sing|Person=3
Sonrası Sonrası Case=Nom|Number=Sing|Number[psor]=Sing|Person=3|Person[psor]=3
Basını Ba Case=Acc|Number=Sing|Number[psor]=Sing|Person=3|Person[psor]=3
Bilgilendirme Bilgilendir Aspect=Perf|Case=Nom|Mood=Ind|Polarity=Pos|Tense=Pres|VerbForm=Vnoun|Voice=Cau
📍Ankara 📍Ankara Case=Nom|Number=Sing|Person=3


In [49]:
# чтобы прогнать на тысяче твитов и записать:

udresults = []
for tweet in data.loc[:1000, 'tweet']:
    for sent in m.process(tweet):
        for word in sent.words:
            udresults.append({'word': word.form, 'lemma': word.lemma, 'pos': word.upostag, 'morph': word.feats})
            
udresults = pd.DataFrame(udresults)
udresults.head()

,word,lemma,pos,morph
0,<root>,<root>,<root>,<root>
1,Geçici,geçici,ADJ,
2,Hükümeti,hükümet,NOUN,Case=Nom|Number=Sing|Number[psor]=Sing|Person=...
3,Dışişleri,dışişleri,NOUN,Case=Nom|Number=Plur|Person=3
4,Bakan,bakan,NOUN,Case=Nom|Number=Sing|Person=3


In [50]:
udresults.to_excel('udturkish.xlsx')